# Exercises

There are three exercises in this notebook:

1. Use the cross-validation method to test the linear regression with different $\alpha$ values, at least three.
2. Implement a SGD method that will train the Lasso regression for 10 epochs.
3. Extend the Fisher's classifier to work with two features. Use the class as the $y$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 

## 1. Cross-validation linear regression

You need to change the variable ``alpha`` to be a list of alphas. Next do a loop and finally compare the results.

In [ ]:
def MSE(A, B):
    return np.square(np.subtract(A, B)).mean()

x = np.array([188, 181, 197, 168, 167, 187, 178, 194, 140, 176, 168, 192, 173, 142, 176]).reshape(-1, 1).reshape(15,1)
y = np.array([141, 106, 149, 59, 79, 136, 65, 136, 52, 87, 115, 140, 82, 69, 121]).reshape(-1, 1).reshape(15,1)

x = np.asmatrix(np.c_[np.ones((15,1)),x])

I = np.identity(2)
alpha = 0.1 # change here
alphas = alpha * np.arange(3)

# add 1-3 line of code here
for alpha in alphas:
    w = np.linalg.inv(x.T*x + alpha * I)*x.T*y
    #w=w.ravel()
    w = np.ravel(w)
    plt.plot(x[:,1], x[:,1]*w[1]+w[0], label=alpha)
    tmp = (x[:,1]*w[1]+w[0] - y)
    error = MSE(y, x[:,1]*w[1]+w[0])
    print(f'alpha {alpha:.1f}: MSE {error:.0f}')

# add 1-3 lines to compare the results
plt.scatter([x[:,1]], y, color='g')
plt.legend()
plt.show()



# 2. Implement based on the Ridge regression example, the Lasso regression.

Please implement the SGD method and compare the results with the sklearn Lasso regression results. 

In [ ]:
rng = np.random.default_rng()

def sgd(x, y, l, alpha=0.001, num=2000):
    w = np.array([[-5.3, 2.3]]) # new weights
    n = y.size
    for _ in range(num):
        j = rng.integers(n)
        pred = x[j] @ w.T
        error = pred - y[j]
        grad = x[j] * error
        grad = 2 * alpha * grad
        grad = grad + l * np.sign(w).reshape(-1,)
        w = w - grad
    return w

N = 30
x = np.arange(N).reshape(-1,1)
x = np.insert(x, 0, 1, axis=1)
w = np.array([[-5, 2.5]])
print('target:', w)
y = x @ w.T
y = y + rng.normal(0, 2, size=y.shape)
shuffle = rng.permutation(N)
y = y[shuffle]
x = x[shuffle]
plt.scatter([x[:,1]], [y], label='org', zorder=3, c='springgreen')


lasso_regression = Lasso(alpha=0.001)
lasso_regression.fit(x[:,1:], y)
w_l = np.stack((lasso_regression.intercept_, lasso_regression.coef_), axis=1)
print('scikit:', w_l)
plt.plot(x[:,1], x @ w_l.T, label=f'scikit', zorder=2, c='silver')

for i, lam in enumerate([0, 0.1, 0.5, 0.8, 0.9, 0.95]):
    print(f'lambda = {lam:.1f}:')
    tmp = sgd(x, y, lam)
    print('  w:', tmp)
    plt.plot(x[:,1].T, x @ tmp.T, label=f'lambda {lam:.1f}', zorder=-i)

plt.legend()
plt.show()

In [ ]:
x = np.array([188, 181, 197, 168, 167, 187, 178, 194, 140, 176, 168, 192, 173, 142, 176]).reshape(-1, 1).reshape(15,1)
y = np.array([141, 106, 149, 59, 79, 136, 65, 136, 52, 87, 115, 140, 82, 69, 121]).reshape(-1, 1).reshape(15,1)

x = np.c_[np.ones((15,1)),x]

I = np.identity(2)
alpha = 0.1 

w2 = np.linalg.inv(x.T @ x + alpha * I) @ x.T @ y # update this line
w2 = w2.ravel()
w = sgd(x, y, 0.5, num=200)
print(w2)
print(w)

plt.scatter(x[:,1], y, label='data')
plt.plot(x[:,1], x @ w2.T, label='ridge', marker='.', c='r')
plt.plot(x[:,1], x @ w.T, label='sgd', marker='o', c='orange')


lasso_regression = Lasso(alpha=0.001)
lasso_regression.fit(x[:,1:], y)
w_l = np.stack((lasso_regression.intercept_, lasso_regression.coef_), axis=1)
print('scikit:', w_l)
plt.plot(x[:,1], x @ w_l.T, label=f'scikit', zorder=2, marker='o', c='silver')
plt.show()

## 3. Extend the Fisher's classifier

Please extend the targets of the ``iris_data`` variable and use it as the $y$.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

iris_data = load_iris()
iris_df = pd.DataFrame(iris_data.data,columns=iris_data.feature_names)
print(iris_df.head())

x = iris_df[['sepal width (cm)', 'petal width (cm)']].values # change here
y = iris_df[['sepal length (cm)', 'petal length (cm)']].values # change here

dataset_size = np.size(x)

mean_x, mean_y = np.mean(x), np.mean(y)

SS_xy = np.sum(y * x) - dataset_size * mean_y * mean_x
SS_xx = np.sum(x * x) - dataset_size * mean_x * mean_x

a = SS_xy / SS_xx
b = mean_y - a * mean_x


y_pred = a * x + b

In [ ]:
plt.scatter(x[:,0], y_pred[:,0])
plt.scatter(x[:,0], y[:,0])
plt.show()

In [ ]:
plt.scatter(x[:,1], y_pred[:,1])
plt.scatter(x[:,1], y[:,1])
plt.show()